In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
import malaya_speech.train.model.vggvox_v1 as vggvox_v1
import malaya_speech.train.model.vggvox_v2 as vggvox_v2
from malaya_speech.train.model import melgan
import numpy as np

In [3]:
resnet = vggvox_v1.model.Resnet1D(
    tf.placeholder(tf.float32, (None, 512, None, 1)), num_class = 7, mode = 'train'
)
resnet.logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


<tf.Tensor 'dense/BiasAdd:0' shape=(?, 7) dtype=float32>

In [4]:
resnet = vggvox_v2.model.Resnet1D(
    tf.placeholder(tf.float32, (None, 257, None, 1)), num_class = 7, mode = 'train'
)
resnet.logits

<tf.Tensor 'prediction/Softmax:0' shape=(?, 7) dtype=float32>

In [5]:
# !wget https://raw.githubusercontent.com/TensorSpeech/TensorFlowTTS/master/examples/melgan/conf/melgan.v1.yaml

In [6]:
import yaml

with open('melgan.v1.yaml', 'r') as stream:
    config = yaml.safe_load(stream)

In [7]:
generator = melgan.model.TFMelGANGenerator(
    melgan.config.MelGANGeneratorConfig(**config['melgan_generator_params']),
    name='melgan-generator',
)
discriminator = melgan.model.TFMelGANMultiScaleDiscriminator(
    melgan.config.MelGANDiscriminatorConfig(**config['melgan_discriminator_params']),
    name='melgan-discriminator',
)

In [8]:
mels_loss = melgan.loss.TFMelSpectrogram()

In [9]:
y = tf.placeholder(tf.float32, (None, None))
x = tf.placeholder(tf.float32, (None, None, 80))
y_hat = generator(x)
d_hat = discriminator(y_hat)
d = discriminator(tf.expand_dims(y, -1))

In [10]:
spect_loss = malaya_speech.train.loss.calculate_2d_loss(
    y, tf.squeeze(y_hat, -1), loss_fn = mels_loss
)

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
y_hat_, loss, d_, d_hat_ = sess.run([y_hat, spect_loss, d_hat, d], 
                                    feed_dict = {x: np.random.uniform(size=(1,200,80)),
                                                 y: np.random.uniform(size=(1,51200))})